In [14]:
import pandas as pd
import sys
import os
import json

In [15]:
RUTA_CARPETA = "Datos/Originales/Datos looks/"  
def cargar_csv_inteligente(nombre_archivo):
    ruta_completa = os.path.join(RUTA_CARPETA, nombre_archivo)
    print(f"Leemos: {nombre_archivo} ...", end=" ")  
    df = pd.read_csv(ruta_completa, sep=None, engine='python')
    if df.shape[1] < 2:
        df = pd.read_csv(ruta_completa, sep=';')   
    df.columns = df.columns.str.strip().str.replace('"', '').str.replace("'", "")
    print(f"✅ OK ({df.shape[0]} filas, {df.shape[1]} cols)")
    return df

df_product = cargar_csv_inteligente('product_2.csv')
df_variant = cargar_csv_inteligente('product_variant.csv')
df_color   = cargar_csv_inteligente('color.csv')
df_size    = cargar_csv_inteligente('size.csv')
df_brand   = cargar_csv_inteligente('brand.csv')
df_pfv     = cargar_csv_inteligente('product_feature_value.csv') 
df_fv      = cargar_csv_inteligente('feature_value.csv')         
df_fvf     = cargar_csv_inteligente('feature_value_family.csv')  
df_feat    = cargar_csv_inteligente('feature.csv')              

# --- 3. PREPARACIÓN DE VARIABLES (RENOMBRADO) ---
if 'id' in df_product.columns: df_product.rename(columns={'id': 'product_id'}, inplace=True)
if 'id' in df_variant.columns: df_variant.rename(columns={'id': 'variant_id'}, inplace=True)
if 'id' in df_color.columns: df_color.rename(columns={'id': 'color_id_ref', 'title': 'color_name'}, inplace=True)
if 'id' in df_size.columns:  df_size.rename(columns={'id': 'size_id_ref', 'title': 'size_name'}, inplace=True)
if 'id' in df_brand.columns: df_brand.rename(columns={'id': 'brand_id_ref', 'title': 'brand_name'}, inplace=True)
if 'id' in df_pfv.columns:   df_pfv.rename(columns={'id': 'pfv_id'}, inplace=True)
if 'id' in df_fv.columns:    df_fv.rename(columns={'id': 'fv_id_ref', 'value': 'feature_value_name'}, inplace=True)
if 'id' in df_feat.columns:  df_feat.rename(columns={'id': 'feat_id_ref', 'title': 'feature_name'}, inplace=True)
df_main = pd.merge(df_product, df_variant, on='product_id', how='left')
df_main['color_id'] = df_main['color_id'].astype(str).str.replace('.0', '', regex=False)
df_color['color_id_ref'] = df_color['color_id_ref'].astype(str)
df_main['size_id'] = df_main['size_id'].astype(str).str.replace('.0', '', regex=False)
df_size['size_id_ref'] = df_size['size_id_ref'].astype(str)
df_main = pd.merge(df_main, df_color, left_on='color_id', right_on='color_id_ref', how='left')
df_main = pd.merge(df_main, df_size, left_on='size_id', right_on='size_id_ref', how='left')
df_main = pd.merge(df_main, df_brand, left_on='brand_id', right_on='brand_id_ref', how='left')
df_feats = pd.merge(df_main, df_pfv, on='product_id', how='left')
df_feats = pd.merge(df_feats, df_fv, left_on='feature_value_id', right_on='fv_id_ref', how='left')
df_feats = pd.merge(df_feats, df_feat, left_on='feature_id', right_on='feat_id_ref', how='left')
df_final = pd.merge(df_feats, df_fvf, left_on='fv_id_ref', right_on='feature_value_id', how='left', suffixes=('', '_fam'))
cols_borrar = [c for c in df_final.columns if '_ref' in c or 'Unnamed' in c]
df_final.drop(columns=cols_borrar, inplace=True, errors='ignore')
print(f"DataFrame final 'df_final': {df_final.shape[0]} filas y {df_final.shape[1]} columnas.")
display(df_final.head(3))

Leemos: product_2.csv ... ✅ OK (2932 filas, 7 cols)
Leemos: product_variant.csv ... ✅ OK (65226 filas, 9 cols)
Leemos: color.csv ... ✅ OK (54 filas, 5 cols)
Leemos: size.csv ... ✅ OK (126 filas, 6 cols)
Leemos: brand.csv ... ✅ OK (163 filas, 4 cols)
Leemos: product_feature_value.csv ... ✅ OK (75347 filas, 3 cols)
Leemos: feature_value.csv ... ✅ OK (2266 filas, 3 cols)
Leemos: feature_value_family.csv ... ✅ OK (34633 filas, 2 cols)
Leemos: feature.csv ... ✅ OK (61 filas, 5 cols)
DataFrame final 'df_final': 14286029 filas y 36 columnas.


,product_id,brand_id,family_id,group,legacy_id_x,title,variant_id,barcode,color_id,legacy_id_y,...,pfv_id,feature_value_id,feature_value_name,feature_id,name,type,multiple,unit,family_id_fam,feature_value_id_fam
0,5f5c4a50-557a-4d53-9e81-cc72fdd18fbf,cfca4b92-12a6-411c-a62a-a874d00aa735,141f32d6-a5af-49bf-b019-fd28354977f1,J1214,\N,Onljade Cardigan knit,fbc2f719-e038-4243-892c-7bd2ca6e80b5,740661,6183657f-3a82-449d-aa6a-a14e96b0e9dd,\N,...,17180364-12d1-423b-b9f4-ed7d121910d2,b5ce1fed-4c38-4920-90b5-42e46f971e97,cardigan,0ccb5374-db8c-4dbb-afae-6b2652af170d,top_type,STRING,f,\N,141f32d6-a5af-49bf-b019-fd28354977f1,b5ce1fed-4c38-4920-90b5-42e46f971e97
1,5f5c4a50-557a-4d53-9e81-cc72fdd18fbf,cfca4b92-12a6-411c-a62a-a874d00aa735,141f32d6-a5af-49bf-b019-fd28354977f1,J1214,\N,Onljade Cardigan knit,fbc2f719-e038-4243-892c-7bd2ca6e80b5,740661,6183657f-3a82-449d-aa6a-a14e96b0e9dd,\N,...,17180364-12d1-423b-b9f4-ed7d121910d2,b5ce1fed-4c38-4920-90b5-42e46f971e97,cardigan,0ccb5374-db8c-4dbb-afae-6b2652af170d,top_type,STRING,f,\N,1b93b30c-833f-45a8-ba69-c21d2e7f4824,b5ce1fed-4c38-4920-90b5-42e46f971e97
2,5f5c4a50-557a-4d53-9e81-cc72fdd18fbf,cfca4b92-12a6-411c-a62a-a874d00aa735,141f32d6-a5af-49bf-b019-fd28354977f1,J1214,\N,Onljade Cardigan knit,fbc2f719-e038-4243-892c-7bd2ca6e80b5,740661,6183657f-3a82-449d-aa6a-a14e96b0e9dd,\N,...,17180364-12d1-423b-b9f4-ed7d121910d2,b5ce1fed-4c38-4920-90b5-42e46f971e97,cardigan,0ccb5374-db8c-4dbb-afae-6b2652af170d,top_type,STRING,f,\N,3087da56-5a27-4e58-aea7-1555c640f232,b5ce1fed-4c38-4920-90b5-42e46f971e97


In [16]:
dffinal = df_final[['product_id','title','season','name_x','lookiero','name_y','feature_value_name','name']]
dffinal = dffinal.drop_duplicates()
dffinal['lookiero'] = dffinal['lookiero'].apply(lambda x: json.loads(x.replace("'", "\""))['size'])
dffinal.to_csv('Datos/Transformados/df_unificado.csv')
dffinal

,product_id,title,season,name_x,lookiero,name_y,feature_value_name,name
0,5f5c4a50-557a-4d53-9e81-cc72fdd18fbf,Onljade Cardigan knit,8,black,XL,BRAND119,cardigan,top_type
6,5f5c4a50-557a-4d53-9e81-cc72fdd18fbf,Onljade Cardigan knit,8,black,XL,BRAND119,ribbed,finishing
36,5f5c4a50-557a-4d53-9e81-cc72fdd18fbf,Onljade Cardigan knit,8,black,XL,BRAND119,two,adventurous
66,5f5c4a50-557a-4d53-9e81-cc72fdd18fbf,Onljade Cardigan knit,8,black,XL,BRAND119,without_closing,closing
96,5f5c4a50-557a-4d53-9e81-cc72fdd18fbf,Onljade Cardigan knit,8,black,XL,BRAND119,cold_season,weather
...,...,...,...,...,...,...,...,...
14285906,98c51c8b-c467-4025-9517-806334bc8680,Sculp Jeans skinny,7,blue_dark,M,BRAND54,skinny,down_part_type
14285909,98c51c8b-c467-4025-9517-806334bc8680,Sculp Jeans skinny,7,blue_dark,M,BRAND54,cotton,composition
14285939,98c51c8b-c467-4025-9517-806334bc8680,Sculp Jeans skinny,7,blue_dark,M,BRAND54,elastane,composition
14285969,98c51c8b-c467-4025-9517-806334bc8680,Sculp Jeans skinny,7,blue_dark,M,BRAND54,smooth,print
